In [1]:
!nvidia-smi

Wed Dec 18 14:24:29 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-32GB           On  |   00000000:06:00.0 Off |                    0 |
| N/A   32C    P0             56W /  300W |    1545MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5'

# Build a RAG System

## Data

In [3]:
exercises = [
    {
      "nama": "Deadlift Konvensional",
      "grup_otot": ["punggung", "hamstring", "gluteus", "trapezius"],
      "deskripsi": "Berdiri dengan kaki selebar pinggul, pegang barbell dengan grip selebar bahu. Angkat barbell dengan meluruskan kaki dan punggung sambil menjaga punggung tetap lurus."
    },
    {
      "nama": "Squat",
      "grup_otot": ["quadriceps", "hamstring", "gluteus"],
      "deskripsi": "Berdiri dengan barbell di bahu, kaki selebar bahu. Turunkan tubuh dengan menekuk lutut hingga paha sejajar dengan lantai, lalu dorong ke atas untuk kembali ke posisi awal."
    },
    {
      "nama": "Bench Press",
      "grup_otot": ["dada", "triceps", "deltoid anterior"],
      "deskripsi": "Berbaring di bench, pegang barbell dengan grip lebih lebar dari bahu. Turunkan barbell ke dada, lalu dorong ke atas hingga lengan lurus."
    },
    {
      "nama": "Pull-up",
      "grup_otot": ["punggung", "biceps", "forearm"],
      "deskripsi": "Gantung pada pull-up bar dengan grip lebih lebar dari bahu. Tarik tubuh ke atas hingga dagu melewati bar, lalu turunkan dengan kontrol."
    },
    {
      "nama": "Military Press",
      "grup_otot": ["deltoid", "triceps", "trapezius"],
      "deskripsi": "Berdiri dengan barbell di depan bahu, angkat barbell ke atas kepala dengan mendorong hingga lengan lurus, lalu turunkan kembali ke posisi awal."
    },
    {
      "nama": "Barbell Row",
      "grup_otot": ["punggung", "biceps", "trapezius"],
      "deskripsi": "Bungkuk dengan punggung lurus, pegang barbell dengan grip selebar bahu. Tarik barbell ke perut, sempitkan belikat, lalu turunkan dengan kontrol."
    },
    {
      "nama": "Leg Press",
      "grup_otot": ["quadriceps", "hamstring", "gluteus"],
      "deskripsi": "Duduk di mesin leg press, letakkan kaki selebar bahu pada platform. Dorong platform hingga kaki hampir lurus, lalu tekuk kembali dengan kontrol."
    },
    {
      "nama": "Dips",
      "grup_otot": ["dada", "triceps", "deltoid anterior"],
      "deskripsi": "Pegang parallel bars, tubuh lurus. Turunkan tubuh dengan menekuk siku hingga bahu sejajar dengan siku, lalu dorong ke atas."
    },
    {
      "nama": "Lat Pulldown",
      "grup_otot": ["punggung", "biceps", "forearm"],
      "deskripsi": "Duduk di mesin lat pulldown, pegang bar dengan grip lebar. Tarik bar ke bawah hingga menyentuh dada bagian atas, lalu kembalikan dengan kontrol."
    },
    {
      "nama": "Romanian Deadlift",
      "grup_otot": ["hamstring", "gluteus", "punggung bawah"],
      "deskripsi": "Berdiri dengan barbell di depan paha, turunkan barbell dengan menekuk pinggul dan menjaga kaki hampir lurus. Kembali ke posisi awal."
    },
    {
      "nama": "Incline Bench Press",
      "grup_otot": ["dada atas", "triceps", "deltoid anterior"],
      "deskripsi": "Berbaring di bench miring, pegang barbell dengan grip lebih lebar dari bahu. Turunkan ke dada atas, lalu dorong ke atas."
    },
    {
      "nama": "Face Pull",
      "grup_otot": ["deltoid posterior", "trapezius", "rotator cuff"],
      "deskripsi": "Gunakan tali di cable machine setinggi bahu, tarik ke arah wajah dengan siku tinggi, fokus pada kontraksi belikat."
    },
    {
      "nama": "Leg Extension",
      "grup_otot": ["quadriceps"],
      "deskripsi": "Duduk di mesin leg extension, angkat beban dengan meluruskan lutut, tahan sebentar di atas, lalu turunkan perlahan."
    },
    {
      "nama": "Hammer Curl",
      "grup_otot": ["biceps", "forearm"],
      "deskripsi": "Berdiri dengan dumbbell di sisi tubuh, telapak tangan menghadap satu sama lain. Angkat dumbbell ke bahu dengan menjaga siku di tempat."
    },
    {
      "nama": "Cable Fly",
      "grup_otot": ["dada", "deltoid anterior"],
      "deskripsi": "Berdiri di tengah cable machine, pegang handle di posisi tinggi. Tarik handle ke depan dengan lengan sedikit ditekuk hingga bertemu di tengah."
    },
    {
      "nama": "Seated Row",
      "grup_otot": ["punggung", "biceps", "trapezius"],
      "deskripsi": "Duduk di mesin row, tarik handle ke perut dengan merapatkan belikat, jaga punggung tegak, lalu kembalikan dengan kontrol."
    },
    {
      "nama": "Calf Raise",
      "grup_otot": ["betis"],
      "deskripsi": "Berdiri di ujung step dengan tumit menggantung, angkat tumit setinggi mungkin, tahan, lalu turunkan di bawah level step."
    },
    {
      "nama": "Tricep Pushdown",
      "grup_otot": ["triceps"],
      "deskripsi": "Berdiri di depan cable machine, pegang bar atau tali di atas. Dorong ke bawah hingga lengan lurus, fokus pada triceps."
    },
    {
      "nama": "Side Lateral Raise",
      "grup_otot": ["deltoid lateral"],
      "deskripsi": "Berdiri dengan dumbbell di sisi tubuh, angkat lengan ke samping hingga sejajar bahu dengan siku sedikit ditekuk."
    },
    {
      "nama": "Leg Curl",
      "grup_otot": ["hamstring"],
      "deskripsi": "Berbaring di mesin leg curl, tekuk lutut untuk mengangkat beban ke arah gluteus, lalu turunkan dengan kontrol."
    },
    {
      "nama": "Decline Bench Press",
      "grup_otot": ["dada bawah", "triceps"],
      "deskripsi": "Berbaring di bench menurun, pegang barbell dengan grip lebih lebar dari bahu. Turunkan ke dada bawah, lalu dorong ke atas."
    },
    {
      "nama": "Preacher Curl",
      "grup_otot": ["biceps"],
      "deskripsi": "Duduk di kursi preacher, letakkan lengan atas di bantalan. Angkat barbell atau dumbbell dengan menekuk siku, lalu turunkan perlahan."
    },
    {
      "nama": "Machine Shoulder Press",
      "grup_otot": ["deltoid", "triceps"],
      "deskripsi": "Duduk di mesin shoulder press, dorong handle ke atas hingga lengan lurus, lalu turunkan ke posisi awal dengan kontrol."
    },
    {
      "nama": "T-Bar Row",
      "grup_otot": ["punggung", "trapezius", "biceps"],
      "deskripsi": "Bungkuk di atas t-bar, pegang handle di bawah beban. Tarik beban ke arah dada dengan merapatkan belikat."
    },
    {
      "nama": "Bulgarian Split Squat",
      "grup_otot": ["quadriceps", "hamstring", "gluteus"],
      "deskripsi": "Letakkan satu kaki di belakang pada bench, kaki depan melangkah ke depan. Turunkan tubuh dengan tekuk lutut depan, lalu dorong naik."
    },
    {
      "nama": "Reverse Fly",
      "grup_otot": ["deltoid posterior", "trapezius"],
      "deskripsi": "Bungkuk dengan dumbbell di tangan, angkat lengan ke samping dengan siku sedikit ditekuk, fokus pada belikat."
    },
    {
      "nama": "Ab Crunch Machine",
      "grup_otot": ["perut"],
      "deskripsi": "Duduk di mesin ab crunch, pegang handle di bahu. Tekuk tubuh ke depan untuk mengkontraksikan otot perut, lalu kembali perlahan."
    },
    {
      "nama": "Wrist Curl",
      "grup_otot": ["forearm"],
      "deskripsi": "Duduk dengan lengan bawah di paha, pergelangan tangan di lutut. Angkat beban dengan menekuk pergelangan tangan ke atas."
    },
    {
      "nama": "Hip Thrust",
      "grup_otot": ["gluteus", "hamstring"],
      "deskripsi": "Duduk di lantai dengan punggung atas di bench, barbell di pinggul. Dorong pinggul ke atas hingga tubuh lurus, lalu turunkan."
    },
    {
      "nama": "Skull Crusher",
      "grup_otot": ["triceps"],
      "deskripsi": "Berbaring di bench, pegang barbell dengan lengan lurus ke atas. Tekuk siku untuk menurunkan barbell ke arah dahi, lalu angkat."
    }
  ]

In [4]:
len(exercises)

30

## Proses Retrieval

In [5]:
def text_to_words(text: str) -> set[str]:
    return set(text.lower().split())
 
def jaccard_similarity(query, text) -> float:
    query_words = text_to_words(query)
    text_words = text_to_words(text)
 
    intersection = query_words.intersection(text_words)
    union = query_words.union(text_words)
 
    return len(intersection) / len(union)

In [6]:
def retrieve(query, documents, k: int = 3):
    scores = [(jaccard_similarity(query, doc), doc) for doc in documents]
    scores.sort(reverse=True, key=lambda x: x[0])
    return scores[:k]

In [7]:
corpus = [
    f'{e["nama"]} ( grup otot: {", ".join(e["grup_otot"])} ). {e["deskripsi"]}'
    for e in exercises
]
corpus[:2]

['Deadlift Konvensional ( grup otot: punggung, hamstring, gluteus, trapezius ). Berdiri dengan kaki selebar pinggul, pegang barbell dengan grip selebar bahu. Angkat barbell dengan meluruskan kaki dan punggung sambil menjaga punggung tetap lurus.',
 'Squat ( grup otot: quadriceps, hamstring, gluteus ). Berdiri dengan barbell di bahu, kaki selebar bahu. Turunkan tubuh dengan menekuk lutut hingga paha sejajar dengan lantai, lalu dorong ke atas untuk kembali ke posisi awal.']

In [8]:
retrieve("angkat barbel untuk triceps", corpus, k=5)

[(0.07407407407407407,
  'Skull Crusher ( grup otot: triceps ). Berbaring di bench, pegang barbell dengan lengan lurus ke atas. Tekuk siku untuk menurunkan barbell ke arah dahi, lalu angkat.'),
 (0.041666666666666664,
  'Wrist Curl ( grup otot: forearm ). Duduk dengan lengan bawah di paha, pergelangan tangan di lutut. Angkat beban dengan menekuk pergelangan tangan ke atas.'),
 (0.04,
  'Leg Extension ( grup otot: quadriceps ). Duduk di mesin leg extension, angkat beban dengan meluruskan lutut, tahan sebentar di atas, lalu turunkan perlahan.'),
 (0.038461538461538464,
  'Calf Raise ( grup otot: betis ). Berdiri di ujung step dengan tumit menggantung, angkat tumit setinggi mungkin, tahan, lalu turunkan di bawah level step.'),
 (0.038461538461538464,
  'Leg Curl ( grup otot: hamstring ). Berbaring di mesin leg curl, tekuk lutut untuk mengangkat beban ke arah gluteus, lalu turunkan dengan kontrol.')]

metode *retrieval* ini masih sederhana karena untuk pengambilan nya sendiri belum mempertimbangkan makna semantik nya.

## Proses Augmented & Generation

In [9]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
    TextStreamer,
    pipeline,
)
 
MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"
 
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
 
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, device_map="auto", torch_dtype=torch.float16
)
 
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 512
generation_config.repetition_penalty = 1.18
generation_config.temperature = 0.0000001
generation_config.do_sample = True

In [10]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
 
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    streamer=streamer,
)

### Proses Augmented

In [11]:
prompt_template = """
Buatkan program latihan untuk melatih kaki - glutes, quadriceps dan hamstrings
Gunakan format:

latihan - set x repetisi

Pilih dari daftar latihan berikut:

{exercises_text}

Pastikan jumlah latihan, set, dan repetisi mengikuti praktik terbaik
untuk pemula
"""

In [13]:
found_exercises = retrieve(prompt_template, corpus, k=3)
found_exercises

[(0.037037037037037035,
  'Deadlift Konvensional ( grup otot: punggung, hamstring, gluteus, trapezius ). Berdiri dengan kaki selebar pinggul, pegang barbell dengan grip selebar bahu. Angkat barbell dengan meluruskan kaki dan punggung sambil menjaga punggung tetap lurus.'),
 (0.03636363636363636,
  'Romanian Deadlift ( grup otot: hamstring, gluteus, punggung bawah ). Berdiri dengan barbell di depan paha, turunkan barbell dengan menekuk pinggul dan menjaga kaki hampir lurus. Kembali ke posisi awal.'),
 (0.03389830508474576,
  'Squat ( grup otot: quadriceps, hamstring, gluteus ). Berdiri dengan barbell di bahu, kaki selebar bahu. Turunkan tubuh dengan menekuk lutut hingga paha sejajar dengan lantai, lalu dorong ke atas untuk kembali ke posisi awal.')]

In [14]:
exercises_text = "\n\n".join([e for _, e in found_exercises])
prompt_text = prompt_template.format(exercises_text=exercises_text)

In [15]:
SYSTEM_PROMPT = """
Anda adalah seorang Pelatih Binaraga ahli dengan pengalaman lebih dari 20 tahun dalam
melatih berbagai tipe atlet. Anda selalu mengikuti prinsip-prinsip dasar saat
merancang program latihan. Setiap sesi latihan harus maksimal 4 jenis latihan.
"""
 
prompt_text = SYSTEM_PROMPT + prompt_text

messages = [
    {"role": "user", "content": prompt_text},
]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 18 Dec 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Anda adalah seorang Pelatih Binaraga ahli dengan pengalaman lebih dari 20 tahun dalam
melatih berbagai tipe atlet. Anda selalu mengikuti prinsip-prinsip dasar saat
merancang program latihan. Setiap sesi latihan harus maksimal 4 jenis latihan.

Buatkan program latihan untuk melatih kaki - glutes, quadriceps dan hamstrings
Gunakan format:

latihan - set x repetisi

Pilih dari daftar latihan berikut:

Deadlift Konvensional ( grup otot: punggung, hamstring, gluteus, trapezius ). Berdiri dengan kaki selebar pinggul, pegang barbell dengan grip selebar bahu. Angkat barbell dengan meluruskan kaki dan punggung sambil menjaga punggung tetap lurus.

Romanian Deadlift ( grup otot: hamstring, gluteus, punggung bawah ). Berdiri dengan barbell di depan paha, turunkan barbell dengan menekuk pinggul dan menjaga kaki hampir lurus.

In [17]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("LazarusNLP/all-indo-e5-small-v4")

In [19]:
prompt_template = """
Buatkan program latihan untuk melatih kaki - glutes, quadriceps dan hamstrings
Gunakan format:

latihan - set x repetisi

Pilih dari daftar latihan berikut:

{exercises_text}

Pastikan jumlah latihan, set, dan repetisi mengikuti praktik terbaik
untuk pemula
"""

prompts = [prompt_template]
prompt_embeddings = embedding_model.encode(prompts, normalize_embeddings=True)
prompt_embeddings.shape

(1, 384)

In [20]:
corpus_embeddings = embedding_model.encode(corpus, normalize_embeddings=True)
similarities = prompt_embeddings @ corpus_embeddings.T
similarities

array([[0.3491032 , 0.549881  , 0.21791556, 0.22441179, 0.28660092,
        0.26272923, 0.5362903 , 0.24394456, 0.23103334, 0.4075893 ,
        0.18959577, 0.19574374, 0.37303728, 0.32258397, 0.15939091,
        0.2583658 , 0.31402504, 0.26763016, 0.16174877, 0.40737325,
        0.17841688, 0.21701388, 0.20281743, 0.21801229, 0.45246014,
        0.20451722, 0.13490015, 0.20137106, 0.4004889 , 0.16263452]],
      dtype=float32)

In [21]:
corpus_embeddings.shape

(30, 384)

In [22]:
similarities.shape

(1, 30)

In [23]:
scores = [(score, document) for score, document in zip(similarities[0], corpus)]
scores = sorted(scores, key=lambda x: x[0], reverse=True)
scores[:5]

[(0.549881,
  'Squat ( grup otot: quadriceps, hamstring, gluteus ). Berdiri dengan barbell di bahu, kaki selebar bahu. Turunkan tubuh dengan menekuk lutut hingga paha sejajar dengan lantai, lalu dorong ke atas untuk kembali ke posisi awal.'),
 (0.5362903,
  'Leg Press ( grup otot: quadriceps, hamstring, gluteus ). Duduk di mesin leg press, letakkan kaki selebar bahu pada platform. Dorong platform hingga kaki hampir lurus, lalu tekuk kembali dengan kontrol.'),
 (0.45246014,
  'Bulgarian Split Squat ( grup otot: quadriceps, hamstring, gluteus ). Letakkan satu kaki di belakang pada bench, kaki depan melangkah ke depan. Turunkan tubuh dengan tekuk lutut depan, lalu dorong naik.'),
 (0.4075893,
  'Romanian Deadlift ( grup otot: hamstring, gluteus, punggung bawah ). Berdiri dengan barbell di depan paha, turunkan barbell dengan menekuk pinggul dan menjaga kaki hampir lurus. Kembali ke posisi awal.'),
 (0.40737325,
  'Leg Curl ( grup otot: hamstring ). Berbaring di mesin leg curl, tekuk lutut 

In [24]:
SYSTEM_PROMPT = """
Anda adalah seorang Pelatih Binaraga ahli dengan pengalaman lebih dari 20 tahun dalam
melatih berbagai tipe atlet. Anda selalu mengikuti prinsip-prinsip dasar saat
merancang program latihan. Setiap sesi latihan harus maksimal 4 jenis latihan.
"""
 
exercises_text = "\n\n".join([e for _, e in scores[:3]])
prompt_text = prompt_template.format(exercises_text=exercises_text)
prompt_text = SYSTEM_PROMPT + prompt_text
 
messages = [
    {"role": "user", "content": prompt_text},
]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
output = llm(prompt)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Berikut adalah contoh program latihan yang dapat Anda gunakan sebagai referensi:

**Latihan Kaki-Glutes**

1. **Squat**
 * Latihan - 8-12 repetisi
 * Posisi:
 + Berdirilah dengan barbell di bahu.
 + Kaki selebar bahu turunkan dengan menekuk lutut sehingga paha sejajar dengan lantai.
 + Dorong ke atas untuk kembali ke posisi awal.
2. **Leg Press**
 * Latihan - 10-15 repetisi
 * Posisi:
 + Duduk di mesin leg press, kaki selebar bahu pada platform.
 + Dorong platform hingga kaki hampir lurus, lalu tekan kembali dengan kontrol.
3. **Bulgarian Split Squat**
 * Latihan - 6-9 repetisi per sisisatu
 + Letakkan satu kaki di belakang pada bench, kaki depan melangkah ke depan.
 + Turunkan tubuh dengan tekuk lutut depan, lalu dorong naik.

Perhatian penting: Pastikan Anda melakukan latihan ini secara kontinuasi, tidak ada istirahat atau berhenti selama beberapa detik. Jangan juga menggunakan gerakan yang salah atau tidak stabil. Selain itu, pastikan Anda memiliki cukup ruang untuk bergerak dengan 